# Plug flow reactor simulation of Thruster

![caption](Graphics/thruster-details.png)


In [1]:
import cantera as ct
import numpy as np

from matplotlib import pyplot as plt
import csv
import pandas as pd

In [2]:
# input file containing the surface reaction mechanism
cti_file = '../RMG-model/cantera/chem_annotated.cti'

cti_file = '../RMG-model/cantera/chem0050.cti'

gas=ct.Solution(cti_file)
surf = ct.Interface(cti_file,'surface1', [gas])

In [3]:
gas()


  gas:

       temperature             300  K
          pressure          101325  Pa
           density         0.81974  kg/m^3
  mean mol. weight         20.1797  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          1905.6        3.845e+04     J
   internal energy      -1.217e+05       -2.456e+06     J
           entropy          7257.7        1.465e+05     J/K
    Gibbs function     -2.1754e+06        -4.39e+07     J
 heat capacity c_p          1030.1        2.079e+04     J/K
 heat capacity c_v          618.03        1.247e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                Ne              1                1         -17.5994
     [  +38 minor]              0                0



In [4]:
print(", ".join(gas.species_names))

Ne, H4N2O2(2), NH2OH(3), HNO3(4), CH3OH(5), H2O(6), N2(7), O2(8), CO2(9), H2(10), CO(11), C2H6(12), CH2O(13), CH3(14), C3H8(15), H(16), C2H5(17), HCO(18), CH3CHO(19), OH(20), C2H4(21), CH4(24), HO2(36), NH2(82), HONO(91), NO2(92), HNOH(94), N2H3(99), H3N2O(193), H2NO2(195), S(196), H2NO3(244), H2NO3(245), H2N2O(381), H2N2O(382), NNDO(385), NNO(394), S(429), S(489)


In [5]:
print(", ".join(surf.species_names))

X(1), HX(22), OX(23), CH3X(25), HOX(26), H2OX(27), CO2X(28), OCX(29), CX(30), CH2X(31), CHX(32), H2NX(211), SX(214), H2NOX(216), H3NOX(217), HNO3X(220), CH3OX(222), CH4OX(223)




This example solves a plug flow reactor problem, with coupled surface and gas chemistry.





In [6]:
# unit conversion factors to SI
cm = 0.01 # m
minute = 60.0  # s

In [7]:
#######################################################################
# Input Parameters for combustor
#######################################################################
mass_flow_rate =  0.5e-3 # kg/s
temperature_c = 400.0  # Initial Temperature in Celsius
pressure = ct.one_atm # constant

length = 1.1 * cm  # Catalyst bed length. 11mm
cross_section_area = np.pi * (0.9*cm)**2  # Catalyst bed area.  18mm diameter circle.

### Catalyst properties. Some are hard to estimate
# if we can, update this lit value or verify the value richard calculated
cat_specific_area = 140 # m2/g
cat_density = 2 / cm**3 # 2 g/m3
print(f"Catalyst density {cat_density :.2e} g/m3")
cat_area_per_vol = cat_specific_area * cat_density  # m2/m3
cat_area_per_vol # m2/m3
print(f"Catalyst area per volume {cat_area_per_vol :.2e} m2/m3")
print()

porosity = 0.38  # Catalyst bed porosity (0.38)
# Al2O3 particles are about 0.7mm diameter

Catalyst density 2.00e+06 g/m3
Catalyst area per volume 2.80e+08 m2/m3



In [8]:
output_filename = 'surf_pfr_output.csv'

# The PFR will be simulated by a chain of 'NReactors' stirred reactors.
NReactors = 2001
dt = 1.0

#####################################################################

temperature_kelvin = temperature_c + 273.15  # convert to Kelvin

# import the gas model and set the initial conditions
gas = ct.Solution(cti_file, 'gas')

# should this be mole fractions or mole fractions?
gas.TPY = temperature_kelvin, pressure, 'H4N2O2(2):0.14, NH2OH(3):0.3, HNO3(4):0.3, CH3OH(5):0.16, H2O(6):0.04'

# import the surface model
surf = ct.Interface(cti_file,'surface1', [gas])
surf.TP = temperature_kelvin, pressure

r_len = length/(NReactors-1) 
r_vol = cross_section_area * r_len * porosity # gas volume

outfile = open(output_filename,'w')
writer = csv.writer(outfile)
writer.writerow(['Distance (mm)', 'T (C)', 'P (atm)'] +
                gas.species_names + surf.species_names + ['alpha'])

# catalyst area in one reactor
cat_area = cat_area_per_vol * r_vol

# Not sure we need the velocity
velocity = mass_flow_rate / (gas.density * cross_section_area)

In [9]:
def report_rates(n=8):
    print("\nHighest net rates of progress, gas")
    for i in np.argsort(abs(gas.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.net_rates_of_progress[i]:8.1g}")
    print("\nHighest net rates of progress, surface")
    for i in np.argsort(abs(surf.net_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.net_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, gas")
    for i in np.argsort(abs(gas.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest forward rates of progress, surface")
    for i in np.argsort(abs(surf.forward_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.forward_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, gas")
    for i in np.argsort(abs(gas.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rates_of_progress[i]:8.1g}")
    print("\nHighest reverse rates of progress, surface")
    for i in np.argsort(abs(surf.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {cat_area_per_vol*surf.reverse_rates_of_progress[i]:8.1g}")

    print(f"\nSurface rates have been scaled by surface/volume ratio {cat_area_per_vol:.1e} m2/m3")
    print("So are on a similar basis of volume of reactor (though porosity not yet accounted for)")
    print(" kmol / m3 / s")
report_rates()


Highest net rates of progress, gas
 41 : H3N2O(193) + OH(20) <=> H4N2O2(2)                   -7e+06
 40 : H3N2O(193) + OH(20) <=> H4N2O2(2)                   -7e+06
 82 : HO2(36) + N2H3(99) <=> H4N2O2(2)                    -1e+02
138 : H2NO3(244) + S(196) <=> H4N2O2(2) + HNO3(4)           -0.3
122 : H2NO3(244) + S(196) <=> H4N2O2(2) + HNO3(4)           -0.3
139 : NO2(92) + OH(20) (+M) <=> HNO3(4) (+M)              -0.002
 84 : H(16) + S(196) <=> H4N2O2(2)                        -1e-05

Highest net rates of progress, surface
  0 : H2O(6) + X(1) <=> H2OX(27)                           8e+07
  1 : NH2OH(3) + X(1) <=> H3NOX(217)                       3e+07
  3 : CH3OH(5) + X(1) <=> CH4OX(223)                       2e+07
  2 : HNO3(4) + X(1) <=> HNO3X(220)                        1e+07
  4 : H4N2O2(2) + X(1) <=> SX(214)                         6e+06
 10 : NH2OH(3) + 2 X(1) <=> H2NOX(216) + HX(22)            4e+03
 15 : CH3OH(5) + 2 X(1) <=> CH3OX(222) + HX(22)            3e+03

Highest forwa

In [10]:
def report_rate_constants(n=8):
    print("\nHighest forward rate constants, gas")
    for i in np.argsort(abs(gas.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.forward_rate_constants[i]:8.1e}")
    print("\nHighest forward rate constants, surface")
    for i in np.argsort(abs(surf.forward_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {surf.forward_rate_constants[i]:8.1e}")
    print("\nHighest reverse rate constants, gas")
    for i in np.argsort(abs(gas.reverse_rate_constants))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {gas.reaction_equation(i):48s}  {gas.reverse_rate_constants[i]:8.1e}")
    print("\nHighest reverse rate constants, surface")
    for i in np.argsort(abs(surf.reverse_rates_of_progress))[-1:-n:-1]: # top n in descending order
        print(f"{i:3d} : {surf.reaction_equation(i):48s}  {surf.reverse_rate_constants[i]:8.1e}")

    print("Units are a combination of kmol, m^3 and s, that depend on the rate expression for the reaction.")
report_rate_constants()


Highest forward rate constants, gas
135 : H2NO3(244) + HCO(18) <=> CH2O(13) + HNO3(4)        3.6e+11
127 : H2NO3(244) + HCO(18) <=> CH2O(13) + HNO3(4)        3.6e+11
102 : H2NO3(245) + HCO(18) <=> CH2O(13) + HNO3(4)        1.8e+11
154 : C2H5(17) + NO2(92) <=> C2H4(21) + HONO(91)         1.4e+11
 27 : HCO(18) + OH(20) <=> CO(11) + H2O(6)               1.1e+11
 15 : H(16) + HCO(18) <=> CO(11) + H2(10)                1.1e+11
 10 : C2H5(17) + H(16) (+M) <=> C2H6(12) (+M)            9.1e+10

Highest forward rate constants, surface
 28 : CHX(32) + HX(22) <=> CH2X(31) + X(1)               3.1e+21
 27 : CH2X(31) + HX(22) <=> CH3X(25) + X(1)              3.1e+21
 29 : CHX(32) + X(1) <=> CX(30) + HX(22)                 3.1e+21
 13 : HX(22) + OX(23) <=> HOX(26) + X(1)                 1.7e+19
 24 : CX(30) + OX(23) <=> OCX(29) + X(1)                 3.7e+18
 26 : CO2X(28) + HX(22) <=> HOX(26) + OCX(29)            2.2e+17
 21 : CH4(24) + HOX(26) + X(1) <=> CH3X(25) + H2OX(27)   4.7e+16

Highest rev

In [11]:
# The plug flow reactor is represented by a linear chain of zero-dimensional
# reactors. The gas at the inlet to the first one has the specified inlet
# composition, and for all others the inlet composition is fixed at the
# composition of the reactor immediately upstream. Since in a PFR model there
# is no diffusion, the upstream reactors are not affected by any downstream
# reactors, and therefore the problem may be solved by simply marching from
# the first to last reactor, integrating each one to steady state.

TDY = gas.TDY
cov = surf.coverages

# create a new reactor
gas.TDY = TDY
r = ct.IdealGasReactor(gas, energy='on')
r.volume = r_vol

# create a reservoir to represent the reactor immediately upstream. Note
# that the gas object is set already to the state of the upstream reactor
upstream = ct.Reservoir(gas, name='upstream')

# create a reservoir for the reactor to exhaust into. The composition of
# this reservoir is irrelevant.
downstream = ct.Reservoir(gas, name='downstream')

# Add the reacting surface to the reactor. The area is set to the desired
# catalyst area in the reactor.
rsurf = ct.ReactorSurface(surf, r, A=cat_area)

# The mass flow rate into the reactor will be fixed by using a
# MassFlowController object.
m = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)

# We need an outlet to the downstream reservoir. This will determine the
# pressure in the reactor. The value of K will only affect the transient
# pressure difference.
v = ct.PressureController(r, downstream, master=m, K=1e-5)

sim = ct.ReactorNet([r])
sim.max_err_test_fails = 24

# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-12
sim.atol = 1.0e-21

sim.verbose = False

print('    distance(mm)     H4N2O2(2)   NH2OH(3)   HNO3(4)  CH3OH(5)  alpha')
for n in range(NReactors):
    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = TDY = r.thermo.TDY

    upstream.syncState()
    sim.reinitialize()
    try:
        # the default is residual_threshold = sim.rtol*10
        sim.advance_to_steady_state(residual_threshold = sim.rtol*1000)
    except ct.CanteraError:
        t = sim.time
        sim.set_initial_time(0)
        gas.TDY = TDY
        r.syncState()
        sim.reinitialize()
        print(f"Couldn't reach {t:.1g} s so going to {0.1*t:.1g} s")
        sim.advance(0.1*t)
        report_rates()
        report_rate_constants()
 
    dist = n * r_len * 1.0e3   # distance in mm
        
    gasHeat = np.dot(gas.net_rates_of_progress, gas.delta_enthalpy) # heat evolved by gas phase reaction
    surfHeat = np.dot(surf.net_rates_of_progress, surf.delta_enthalpy) # heat evolved by surf phase reaction 
    alpha = gasHeat/surfHeat #ratio of gas heat evolved to surface heat evolved.

    if not n % 10:
        print('    {:10f}  {:10f}  {:10f}  {:10f} {:10f}  {:5.1e}'.format(dist, *gas['H4N2O2(2)','NH2OH(3)','HNO3(4)','CH3OH(5)'].X, alpha ))

    # write the gas mole fractions and surface coverages vs. distance
    writer.writerow([dist, r.T - 273.15, r.thermo.P/ct.one_atm] +
                    list(gas.X) + list(surf.coverages) + [alpha])
    
    #report_rates()
    #report_rate_constants()

outfile.close()
print("Results saved to '{0}'".format(output_filename))

    distance(mm)     H4N2O2(2)   NH2OH(3)   HNO3(4)  CH3OH(5)  alpha
      0.000000    0.100527    0.407970    0.222848   0.224162  1.6e+08
Couldn't reach 4e-11 s so going to 4e-12 s


CanteraError: 
***********************************************************************
CanteraError thrown by CVodesIntegrator::integrate:
CVodes error encountered. Error code: -4
At t = 1.11999e-12 and h = 3.51893e-21, the corrector convergence test failed repeatedly or with |h| = hmin.

Components with largest weighted error estimates:
46: -1664.48
44: 1.75906
47: 0.00586492
26: 0.000252626
30: -2.55471e-05
18: -7.94603e-06
52: 6.63304e-07
50: -3.60209e-07
1: -1.94229e-07
0: -5.7764e-08
***********************************************************************


In [ ]:
sim.time

In [ ]:
gas.TDY = TDY
r.syncState()
r.thermo.T

In [ ]:
r.thermo.X - gas.X

In [ ]:
rsurf.kinetics.net_rates_of_progress

In [ ]:
surf.net_rates_of_progress

In [ ]:
gas.TDY

In [ ]:
r.thermo.TDY

In [ ]:
report_rate_constants()

In [ ]:
sim.verbose


In [ ]:
sim.component_name(40)

In [ ]:
gas.species_index('S(429)')

In [ ]:
plt.barh(np.arange(len(gas.net_rates_of_progress)),gas.net_rates_of_progress)

In [ ]:
gas.T

In [ ]:
gas.T

In [ ]:
data = pd.read_csv(output_filename)
data

In [ ]:
data['T (C)'].plot()

In [ ]:
data[['H4N2O2(2)', 'CH3OH(5)']].plot()

In [ ]:
list(data.columns)[:4]

In [ ]:
specs = list(data.columns)
specs = specs[4:]
specs

In [ ]:
data[specs[1:5]].plot()

for i in range(0,len(specs),10):
    data[specs[i:i+10]].plot()

In [ ]:
gas.species('NO(49)').composition

In [ ]:
data['NO(49)'].plot()

In [ ]:
(data[specs].max()>0.01)

In [ ]:
data.loc[0]